In [7]:
import nltk
import pymorphy2
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus.reader import WordListCorpusReader
import itertools
import pandas as pd
import string 
from sklearn import tree

###

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

###
#mysvc = LinearSVC()
# X is list of poscorpus vectors. One for each sentence
# Y is the correct parts of speach also passed thru the countvectorizer.
#mysvc.fit(X, Y)
#######

#corpus = [
#'Из открытой двери сильно дуло в ноги.',
#'Воткнув палец в дуло солдатского ружья, он мог поднять его и держать горизонтально.']

#locs = [5, 4]

#Y = [
#'verb',
#'noun' # for example
#]


test = pd.read_csv ("/Users/biatris/Desktop/dulo_test_set.tsv", sep = "\t")
train = pd.read_csv ("/Users/biatris/Desktop/dulo_train_set.tsv", sep = "\t")
fulldata = pd.concat([train, test])
print(fulldata)

    word    word_id                                               sent  start  \
0   дуло  дуло_noun  Выхватив пистолет, он падает на колени, приста...     53   
1   дуло  дуло_noun  Данзас подал Пушкину другой пистолет: дуло пер...     39   
2   дуло  дуло_noun  Он выстрелил, сунув дуло "ТТ" под мышку, в стену.     21   
3   дуло  дуло_noun  Затем ощутил на себе взгляд, холодный и твёрды...     54   
4   дуло  дуло_noun  Шурка вскочил с дивана и за дуло выволок ружьё...     29   
5   дуло  дуло_noun  Обиду стёрла кровь, и ты, ты думаешь, по нём в...     71   
6   дуло  дуло_noun  Андрей открыл глаза и увидел наставленное на н...     51   
7   дуло  дуло_noun  Соседи проснулись и стали давать советы самоуб...     70   
8   дуло  дуло_noun  И люди заторопились, рядом лежат убитые и недо...     95   
9   дуло  дуло_noun  Он кричал, свистел, но неизвестный пёр на него...     65   
10  дуло  дуло_noun  Вот и выходит: ты тут стоишь, а смерть твоя в ...     92   
11  дуло  дуло_noun  Сержант

In [6]:
test['sent'] = test['sent'].apply( lambda s : s.lower() )
train['sent'] = train['sent'].apply( lambda s : s.lower() )
wtest = test [test["word"] == "дуло"]
wtrain = train [train["word"] == "дуло"]
Xinit = wtrain["sent"].values

Yinit = wtrain["word_id"]
Yinit = Yinit.apply(lambda x: x.replace("дуло_", "", 1).upper())
#d = {"NOUN" : 0, "VERB" : 1}
allpos = ["NOUN", "ADJF", "ADJS", "COMP", "VERB", "INFN", "PRTF", "PRTS", "GRND", "NUMR", 
          "ADVB", "NPRO", "PRED", "PREP", "CONJ", "PRCL", "INTJ", "NONE"]
Yinit = Yinit.apply(lambda x: allpos.index(x)).values



Xinit_test = wtest["sent"].values
Yinit_test = wtest["word_id"]
Yinit_test = Yinit_test.apply(lambda x: x.replace("дуло_", "", 1).upper())
Yinit_test = Yinit_test.apply(lambda x: allpos.index(x)).values


print(Xinit_test)
print(Yinit_test)
#print(wtest) 

    word    word_id                                               sent  start  \
0   дуло  дуло_noun  Выхватив пистолет, он падает на колени, приста...     53   
1   дуло  дуло_noun  Данзас подал Пушкину другой пистолет: дуло пер...     39   
2   дуло  дуло_noun  Он выстрелил, сунув дуло "ТТ" под мышку, в стену.     21   
3   дуло  дуло_noun  Затем ощутил на себе взгляд, холодный и твёрды...     54   
4   дуло  дуло_noun  Шурка вскочил с дивана и за дуло выволок ружьё...     29   
5   дуло  дуло_noun  Обиду стёрла кровь, и ты, ты думаешь, по нём в...     71   
6   дуло  дуло_noun  Андрей открыл глаза и увидел наставленное на н...     51   
7   дуло  дуло_noun  Соседи проснулись и стали давать советы самоуб...     70   
8   дуло  дуло_noun  И люди заторопились, рядом лежат убитые и недо...     95   
9   дуло  дуло_noun  Он кричал, свистел, но неизвестный пёр на него...     65   
10  дуло  дуло_noun  Вот и выходит: ты тут стоишь, а смерть твоя в ...     92   
11  дуло  дуло_noun  Сержант

In [2]:
tokenizecorp = []
#for c in corpus:
#    tokenizecorp.append(nltk.word_tokenize(c))
locs = []
for index, row in wtrain.iterrows(): #loop over to find word location not character
    q = nltk.word_tokenize(row["sent"])
    print(q)
    tokenizecorp.append(q)
    start = row["start"]-1
    stop = row["stop"]-1
    word = row["sent"][start:stop] #extract the word
    print(word)
    locs.append(q.index(word))
   
#convert to parts of speech
morph = pymorphy2.MorphAnalyzer()
morph.parse(repr(tokenizecorp))
Tag = morph.TagClass

# poscorp.append([p.tag.POS for p in morph.parse(token)]) #list of parts of speech eg
#['NOUN', 'VERB', ....]
#for word in tokenized_homonyms:
# print(word, [p.tag.POS for p in morph.parse(word)])
#poscorp should look = [['NOUN', 'VERB', ....], ['ADJ', 'VERB', ....],....]

poscorp = []

for s in tokenizecorp:
    tokenizedsen = []
    for token in s:
        if token not in list(string.punctuation) + ["''", "``", "―", "«", "»", "тт",]:
            q = list(set( [p.tag.POS for p in morph.parse(token)] ))
            tokenizedsen.append(q)
            poscorp.append( tokenizedsen )
            print(token, q)


#print(tokenizecorp)
#print(locs)


['выхватив', 'пистолет', ',', 'он', 'падает', 'на', 'колени', ',', 'приставляет', 'дуло', 'к', 'виску', 'и', 'грозит', 'застрелиться', 'на', 'её', 'глазах', ',', 'если', 'она', 'сию', 'же', 'минуту', 'не', 'согласится', 'на', 'то', ',', 'о', 'чём', 'её', 'умолял', 'приёмный', '``', 'отец', "''", '.']
дуло
['данзас', 'подал', 'пушкину', 'другой', 'пистолет', ':', 'дуло', 'первого', 'при', 'падении', 'забилось', 'снегом', '.']
дуло
['он', 'выстрелил', ',', 'сунув', 'дуло', '``', 'тт', "''", 'под', 'мышку', ',', 'в', 'стену', '.']
дуло
['затем', 'ощутил', 'на', 'себе', 'взгляд', ',', 'холодный', 'и', 'твёрдый', ',', 'как', 'дуло', '.']
дуло
['шурка', 'вскочил', 'с', 'дивана', 'и', 'за', 'дуло', 'выволок', 'ружьё', 'из', 'прихожей', '.']
дуло
['обиду', 'стёрла', 'кровь', ',', 'и', 'ты', ',', 'ты', 'думаешь', ',', 'по', 'нём', 'вздыхая', ',', 'что', 'я', 'приставлю', 'дуло', '(', 'я', '!', ')', 'к', 'виску', '?']
дуло
['андрей', 'открыл', 'глаза', 'и', 'увидел', 'наставленное', 'на', 'него'

дуло
['в', 'домике', 'пахло', 'паром', 'и', 'смолой', ',', 'дуло', 'из', 'щелей', ':', 'он', 'был', 'построен', 'из', 'непросохшего', 'леса', '.']
дуло
['когда', 'от', 'очереди', ',', 'гудящей', ',', 'сующей', 'ему', 'продаттестаты', ',', 'он', 'был', 'отделён', 'закрытым', 'окном', 'и', 'решёткой', 'и', 'только', 'малая', 'форточка', 'оставлена', 'для', 'сующихся', 'рук', '―', 'чичишев', 'бывал', 'очень', 'твёрд', ',', 'кричал', 'на', 'бойцов', ',', 'руки', 'отталкивал', 'и', 'форточку', 'прикрывал', ',', 'чтоб', 'не', 'дуло', '.']
дуло
['там', ',', 'в', 'двух', 'небольших', 'комнатах', ',', 'теснилось', 'двадцать', 'пять', '―', 'тридцать', 'человек', '.', 'из-под', 'полов', 'дуло', '.', 'в', 'избушке', 'поставили', 'чугунную', 'печку', ',', 'которую', 'раскаляли', 'докрасна', '.']
дуло
['он', 'был', 'не', 'причесан', ',', 'борода', 'торчала', ',', 'ворот', 'рубашки', 'не', 'застегнут', ',', 'хотя', 'в', 'комнате', 'было', 'холодно', ',', 'дуло', 'от', 'окна', '.']
дуло
['и', 'не', 'п

убийства ['NOUN']
не ['PRCL']
отпугивает ['VERB']
не ['PRCL']
попадает ['VERB']
в ['NOUN', 'PREP']
поле ['NOUN']
зрения ['NOUN']
даже ['CONJ', 'PRCL']
впоследние ['ADJF', 'NOUN']
секунды ['NOUN']
жизни ['NOUN']
человек ['NOUN']
не ['PRCL']
хочет ['VERB']
пренебречь ['INFN']
комфортом ['NOUN']
и ['INTJ', 'NOUN', 'CONJ', 'PRCL']
сатана ['NOUN']
склонившись ['GRND']
над ['PREP']
рублевкой ['NOUN']
стал ['VERB']
запихивать ['INFN']
дуло ['VERB', 'NOUN']
нагана ['NOUN']
ему ['NPRO']
в ['NOUN', 'PREP']
рот ['NOUN']
я ['NPRO']
действительно ['CONJ', 'PRCL', 'ADJS', 'ADVB']
отвык ['VERB']
что ['NPRO', 'CONJ', 'PRCL', 'ADVB']
каждая ['ADJF']
мразь ['NOUN']
может ['VERB', 'CONJ']
приставить ['INFN']
тебе ['NPRO']
дуло ['VERB', 'NOUN']
к ['NOUN', 'PREP', 'ADVB']
виску ['NOUN']
обыскать ['INFN']
унизить ['INFN']
.. [None]
он ['NPRO']
втискивает ['VERB']
дуло ['VERB', 'NOUN']
меж ['NOUN', 'PREP']
губ ['NOUN']
и ['INTJ', 'NOUN', 'CONJ', 'PRCL']
медленно ['ADJS', 'ADVB']
спускает ['VERB']
курок ['NOU

и ['INTJ', 'NOUN', 'CONJ', 'PRCL']
с ['NOUN', 'PREP']
неясным ['ADJF']
страхом ['NOUN']
заглянул ['VERB']
в ['NOUN', 'PREP']
дуло ['VERB', 'NOUN']
на ['INTJ', 'PRCL', 'PREP']
лице ['NOUN']
борменталя ['NOUN']
а ['INTJ', 'CONJ', 'PRCL']
затем ['ADVB']
на ['INTJ', 'PRCL', 'PREP']
филиппа ['NOUN']
филипповича ['NOUN']
башня ['NOUN']
поворачивается ['VERB']
сама ['ADJF']
собой ['NPRO']
наводя ['GRND']
на ['INTJ', 'PRCL', 'PREP']
город ['NOUN']
насквозь ['ADVB']
пустое ['ADJF']
сияющее ['PRTF']
внутри ['PREP', 'ADVB']
зеркальными ['ADJF']
нарезами ['NOUN']
дуло ['VERB', 'NOUN']
мелькнула ['VERB']
в ['NOUN', 'PREP']
глазах ['NOUN']
черная ['ADJF']
нашивка ['NOUN']
и ['INTJ', 'NOUN', 'CONJ', 'PRCL']
ледяным ['ADJF']
поцелуем ['VERB', 'NOUN']
коснулось ['VERB']
виска ['NOUN']
револьверное ['ADJF']
дуло ['VERB', 'NOUN']
за ['PREP']
кустом ['NOUN']
мелькнуло ['VERB']
дуло ['VERB', 'NOUN']
винтовки ['NOUN']
и ['INTJ', 'NOUN', 'CONJ', 'PRCL']
над ['PREP']
головой ['NOUN']
его ['NPRO', 'ADJF']
прос

сквозняк ['NOUN']
то ['CONJ', 'PRCL', 'ADJF']
ли ['NOUN', 'CONJ', 'PRCL']
из ['NOUN', 'PREP']
незанятых ['ADJF']
комнат ['NOUN']
то ['CONJ', 'PRCL', 'ADJF']
ли ['NOUN', 'CONJ', 'PRCL']
от ['PREP']
окон ['NOUN']
дуло ['VERB', 'NOUN']
дуло… [None]
спали ['VERB']
не ['PRCL']
раздеваясь ['GRND']
в ['NOUN', 'PREP']
пальто ['NOUN']
шапках ['NOUN']
а ['INTJ', 'CONJ', 'PRCL']
то ['CONJ', 'PRCL', 'ADJF']
и ['INTJ', 'NOUN', 'CONJ', 'PRCL']
в ['NOUN', 'PREP']
валенках ['NOUN']
от ['PREP']
этих ['ADJF']
ночей ['NOUN']
можно ['PRED']
было ['VERB', 'PRCL']
сойти ['INFN']
с ['NOUN', 'PREP']
ума ['NOUN']
из ['NOUN', 'PREP']
раскрытого ['ADJF', 'PRTF']
окна ['NOUN']
тонко ['ADJS', 'ADVB']
дуло ['VERB', 'NOUN']
прохладой ['NOUN']
одуряюще ['ADJS', 'ADVB']
пахла ['VERB']
молодая ['ADJF', 'NOUN']
листва ['NOUN']
скрипели ['VERB']
грачи ['NOUN']
а ['INTJ', 'CONJ', 'PRCL']
может ['VERB', 'CONJ']
быть ['INFN']
звёзды ['NOUN']
уж ['NOUN', 'PRCL', 'ADVB']
который ['ADJF']
день ['VERB', 'NOUN']
дуло ['VERB', 'N

в ['NOUN', 'PREP']
февральские ['ADJF']
морозы ['NOUN']
когда ['CONJ', 'ADVB']
дуло ['VERB', 'NOUN']
из ['NOUN', 'PREP']
всех ['ADJF']
щелей ['NOUN']
раздевалась ['VERB']
она ['NPRO']
обязательно ['ADJS', 'ADVB']
догола ['ADVB']
раскидывая ['GRND']
одежду ['NOUN']
по ['PREP']
комнате ['NOUN']
во ['PREP']
время ['NOUN']
объятий ['NOUN']
оставляла ['VERB']
гореть ['INFN']
лампу ['NOUN']
и ['INTJ', 'NOUN', 'CONJ', 'PRCL']
сердилась ['VERB']
если ['CONJ']
свечниковпытался ['VERB']
ее ['NPRO', 'ADJF']
потушить ['INFN']
извини ['VERB']
я ['NPRO']
не ['PRCL']
женщина ['NOUN']
востока ['NOUN']
чтобы ['CONJ', 'PRCL']
любить ['INFN']
тебя ['NPRO']
в ['NOUN', 'PREP']
полной ['ADJF']
темноте ['NOUN']
заболел ['VERB']
ковалев ['NOUN']
не ['PRCL']
краснухой ['NOUN']
а ['INTJ', 'CONJ', 'PRCL']
корью ['NOUN']
из-за ['PREP']
того ['ADJF', 'NOUN']
что ['NPRO', 'CONJ', 'PRCL', 'ADVB']
в ['NOUN', 'PREP']
этой ['ADJF']
тюрьме ['NOUN']
всюду ['ADVB']
стояли ['VERB']
кондиционеры ['NOUN']
и ['INTJ', 'NOUN', 

случая ['NOUN', 'GRND']
зайти ['INFN']
к ['NOUN', 'PREP', 'ADVB']
ней ['NPRO']
не ['PRCL']
будет ['VERB']
да ['INTJ', 'CONJ', 'PRCL']
и ['INTJ', 'NOUN', 'CONJ', 'PRCL']
на ['INTJ', 'PRCL', 'PREP']
набережной ['NOUN']
сильно ['ADJS', 'ADVB']
дуло ['VERB', 'NOUN']
но ['INTJ', 'CONJ']
я ['NPRO']
бы ['PRCL']
сказал ['VERB']
что ['NPRO', 'CONJ', 'PRCL', 'ADVB']
проявился ['VERB']
уже ['COMP', 'NOUN', 'PRCL', 'ADVB']
какой-то ['ADJF']
сквознячок ['NOUN']
вроде ['PRCL', 'PREP']
бы ['PRCL']
ощущались ['VERB']
какие-то ['ADJF']
щели ['NOUN']
сквозь ['PREP']
которые ['ADJF']
дуло ['VERB', 'NOUN']
— [None]
хотите ['VERB']
я ['NPRO']
лучше ['COMP', 'PRCL']
закрою ['VERB', 'NOUN']
форточку ['NOUN']
чтобы ['CONJ', 'PRCL']
не ['PRCL']
дуло ['VERB', 'NOUN']
острым ['ADJF']
первобытным ['ADJF']
холодом ['NOUN']
дуло ['VERB', 'NOUN']
с ['NOUN', 'PREP']
речного ['ADJF']
льда ['NOUN']
окатывало ['VERB']
густым ['ADJF']
паром ['NOUN']
из ['NOUN', 'PREP']
дымящихся ['PRTF']
внизу ['PREP', 'ADVB']
огромных [

ни ['CONJ', 'PRCL']
даже ['CONJ', 'PRCL']
проточной ['ADJF']
воды ['NOUN']


In [3]:
poscorpus = []
poscorpus.append( " ".join( allpos ) )
for step in zip( locs, poscorp ):
    loc = step[0]
    tokenizedsen = step[1]
    prev2 = tokenizedsen[ max(loc - 1, 0 ) : loc ]
    next2 = tokenizedsen[ loc : min(loc + 1, len(tokenizedsen)-1) ]
    print(prev2)
    
    print(next2)
    print()
    #print(" ".join(list(itertools.chain.from_iterable(prev2))) )
    #print(" ".join(list(itertools.chain.from_iterable(next2))) )

    poscorpus.append( " ".join(list(itertools.chain.from_iterable(prev2))) + " " + " ".join(list(itertools.chain.from_iterable(next2))) )
    
print(poscorp)
print('KIWI')
print(poscorpus)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(poscorpus)
X = X.toarray()
print(vectorizer.get_feature_names())

[['NOUN', 'PREP', 'ADVB']]
[['NOUN']]

[['NOUN']]
[['VERB']]

[['VERB']]
[['INTJ', 'PRCL', 'PREP']]

[['INTJ', 'NOUN', 'CONJ', 'PRCL']]
[['VERB']]

[['NOUN']]
[['VERB']]

[['CONJ']]
[['NPRO']]

[['VERB', 'NOUN']]
[['NOUN', 'PREP', 'ADVB']]

[['NPRO', 'ADJF']]
[['NOUN']]

[['NPRO']]
[['ADJF']]

[['INTJ', 'PRCL', 'PREP']]
[['NPRO', 'ADJF']]

[['NOUN']]
[['PRCL']]

[['VERB']]
[['VERB', 'NOUN']]

[['NPRO']]
[['ADJF']]

[['INFN']]
[['INTJ', 'PRCL', 'PREP']]

[['NOUN']]
[['NPRO']]

[['NOUN']]
[['CONJ']]

[['INTJ', 'NOUN', 'CONJ', 'PRCL']]
[['VERB']]

[['INTJ', 'NOUN', 'CONJ', 'PRCL']]
[['VERB']]

[['NPRO']]
[['VERB']]

[['NOUN']]
[['CONJ']]

[['VERB', 'NOUN']]
[['NOUN', 'PREP', 'ADVB']]

[['NOUN']]
[['VERB']]

[['NOUN']]
[['VERB']]

[['NOUN']]
[['NPRO']]

[['NOUN', 'PREP', 'ADVB']]
[['NOUN']]

[['NOUN']]
[['NPRO']]

[['NOUN']]
[['VERB']]

[['INTJ', 'PRCL', 'PREP']]
[['NOUN']]

[['VERB']]
[['INFN']]

[]
[['GRND']]

[['VERB', 'NOUN']]
[['NOUN', 'PREP', 'ADVB']]

[['NOUN']]
[['VERB']]

[['NOUN'

In [4]:
tokenizecorp = []
#for c in corpus:
#    tokenizecorp.append(nltk.word_tokenize(c))
locs = []
for index, row in wtest.iterrows(): #loop over to find word location not character
    q = nltk.word_tokenize(row["sent"])
    tokenizecorp.append(q)
    #word = row["word"]
    #start = row["sent"].find(row["word"])
    #stop = start + len(word) 
    start = row["start"]-1
    stop = row["stop"]-1
    word = row["sent"][start:stop] #extract the word
    #print(word, start, stop)
    locs.append(q.index(word))
print(locs)    

#create tokened versions of corpus
#tokenizecorp = [['Из' , 'открытой', .....], ['Воткнув', 'палец'],....]
#convert to parts of speech
morph = pymorphy2.MorphAnalyzer()
morph.parse(repr(tokenizecorp))
Tag = morph.TagClass

# poscorp.append([p.tag.POS for p in morph.parse(token)]) #list of parts of speech eg
#['NOUN', 'VERB', ....]
#for word in tokenized_homonyms:
# print(word, [p.tag.POS for p in morph.parse(word)])
#poscorp should look = [['NOUN', 'VERB', ....], ['ADJ', 'VERB', ....],....]

poscorp = []

for s in tokenizecorp:
    tokenizesen = []
    for token in s:
        if token not in string.punctuation:
            
            tokenizesen.append(list(set( [p.tag.POS for p in morph.parse(token)] )))
            poscorp.append( tokenizesen )

print(tokenizecorp)
print(locs)

poscorpus = []
poscorpus.append( " ".join( allpos ) )
for step in zip( locs, poscorp ):
    loc = step[0]
    tokenizedsen = step[1]
    prev2 = tokenizedsen[ max(loc - 1, 0 ) : loc ]
    next2 = tokenizedsen[ loc : min(loc + 1, len(tokenizedsen)-1) ]
    print(prev2)
    print(next2)
    #print(" ".join(list(itertools.chain.from_iterable(prev2))) )
    #print(" ".join(list(itertools.chain.from_iterable(next2))) )

    poscorpus.append( " ".join(list(itertools.chain.from_iterable(prev2))) + " " + " ".join(list(itertools.chain.from_iterable(next2))) )
    
#print(tokenizecorp)
#print(poscorp)
#print(poscorpus)

vectorizer = CountVectorizer()
X_test = vectorizer.fit_transform(poscorpus)
X_test = X_test.toarray()
print(vectorizer.get_feature_names())
print(X_test)

[4, 11, 8, 12, 0, 21, 8, 10, 3, 4, 11, 33, 21, 32, 10, 10, 8, 22, 45, 3, 7, 16, 5, 16, 19, 7, 20, 2, 9, 6, 8, 9, 9, 19, 8, 12, 10, 33, 1, 11, 18]
[['из', 'щелей', 'все', 'равно', 'дуло', ',', 'но', 'это', 'оставим', 'на', 'потом', '.'], ['заботится', 'о', 'дверях', ',', 'щелях', 'и', 'проч', '.', ',', 'чтоб', 'не', 'дуло', '.', 'затворы', ',', 'замки', ',', 'засовы', ':', 'все', 'скрепы', 'герметические', '.'], ['слава', 'богу', ',', 'ночью', 'не', 'сыпало', 'и', 'не', 'дуло', ',', 'снег', 'чистить', 'нет', 'нужды', '.'], ['я', 'сел', 'было', 'на', 'пол', ',', 'как', 'сидел', 'василек', ',', 'но', 'понизу', 'дуло', '.'], ['дуло', 'кислятиной', '.', 'в', 'полумраке', 'с', 'потолка', 'и', 'с', 'труб', 'свисали', 'клочья', 'то', 'ли', 'асбестовых', 'прокладок', ',', 'то', 'ли', 'летучих', 'мышей', '.'], ['под', 'черными', 'ярами', 'правого', 'берега', 'было', 'тихо', ',', 'на', 'левом', ',', 'луговом', ',', '―', 'вьюжило', ';', 'по', 'льду', 'замерзшей', 'широкой', 'реки', 'дуло', ',', 'с

In [5]:
##############################################################
#X_test = X_test[:1] 
print(X_test)
print(Yinit_test)
mysvc = LinearSVC(verbose = 2)
mydt = tree.DecisionTreeClassifier()
rX = X[1:]
rX_test = X_test[1:]
print(rX)
print(Yinit)
mysvc.fit(rX, Yinit)
mydt.fit(rX, Yinit)
print(mysvc.score( rX_test, Yinit_test ) )
print (mydt.score( rX_test, Yinit_test ) )


[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1]
 [0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [1 1 2 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 1 0 2 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0